In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
df = pd.read_csv('Wrangled.csv')

C:\Users\veron\AppData\Local\Temp\ipykernel_12452\2296681516.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Wrangled.csv')


In [3]:
df.head()

,Unnamed: 0,Rfrg_Prvdr_Geo_Lvl,Rfrg_Prvdr_Geo_Cd,Rfrg_Prvdr_Geo_Desc,BETOS_Lvl,BETOS_Cd,BETOS_Desc,HCPCS_Cd,HCPCS_Desc,Suplr_Rentl_Ind,Tot_Rfrg_Prvdrs,Tot_Suplrs,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Avg_Suplr_Sbmtd_Chrg,Avg_Suplr_Mdcr_Alowd_Amt,Avg_Suplr_Mdcr_Pymt_Amt,Avg_Suplr_Mdcr_Stdzd_Amt
0,0,National,6.0,National,Drugs and Nutritional Products,O1C,Enteral and parenteral,B4034,"Enteral feeding supply kit; syringe fed, per d...",N,28183,1745,33451.0,171371,4627387,58.266520,3.718512,2.918066,3.047039
1,1,National,6.0,National,Drugs and Nutritional Products,O1C,Enteral and parenteral,B4035,"Enteral feeding supply kit; pump fed, per day,...",N,27342,1758,39110.0,271836,7050743,75.495042,6.433860,5.067509,5.552222
2,2,National,6.0,National,Drugs and Nutritional Products,O1C,Enteral and parenteral,B4036,"Enteral feeding supply kit; gravity fed, per d...",N,6166,933,7171.0,41181,1116382,75.172639,5.036933,3.959289,4.100195
3,3,National,6.0,National,Drugs and Nutritional Products,O1C,Enteral and parenteral,B4081,Nasogastric tubing with stylet,N,14,12,13.0,20,33,113.736061,17.292727,13.833939,13.579697
4,4,National,6.0,National,Drugs and Nutritional Products,O1C,Enteral and parenteral,B4082,Nasogastric tubing without stylet,N,16,16,16.0,21,43,99.110930,12.532093,9.502326,9.463488


In [4]:
df = df[['HCPCS_Cd', 'Tot_Rfrg_Prvdrs', 'Tot_Suplrs', 'Avg_Suplr_Sbmtd_Chrg']]

In [5]:
df.head()

,HCPCS_Cd,Tot_Rfrg_Prvdrs,Tot_Suplrs,Avg_Suplr_Sbmtd_Chrg
0,B4034,28183,1745,58.266520
1,B4035,27342,1758,75.495042
2,B4036,6166,933,75.172639
3,B4081,14,12,113.736061
4,B4082,16,16,99.110930


In [6]:
tot_rfrg_prvdrs_max = df['Tot_Rfrg_Prvdrs'].max()
df.loc[df['Tot_Rfrg_Prvdrs'] == tot_rfrg_prvdrs_max]

,HCPCS_Cd,Tot_Rfrg_Prvdrs,Tot_Suplrs,Avg_Suplr_Sbmtd_Chrg
592,A4253,233762,43511,61.932717


In [7]:
blood_glucose_test = df[df.HCPCS_Cd == 'A4253']
blood_glucose_test.T

,592,1999,2624,3186,3987,4884,5882,6642,7284,7821,...,38072,38787,39428,39723,39856,40037,40109,40154,40206,40228
HCPCS_Cd,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253,...,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253,A4253
Tot_Rfrg_Prvdrs,233762,3093,509,4387,2258,22583,3630,2900,821,649,...,1872,4895,470,41,1093,59,8,7,7,9
Tot_Suplrs,43511,1406,204,2262,924,6656,1746,1825,543,853,...,843,2288,251,17,244,42,68,47,51,30
Avg_Suplr_Sbmtd_Chrg,61.932717,56.55811,64.369821,66.860627,49.084897,66.023182,58.680633,69.086442,69.558235,59.030813,...,62.519122,66.743051,62.289233,72.101268,45.837617,62.153743,56.325625,65.26864,63.750638,63.44035


In [8]:
blood_glucose_test.shape

(59, 4)

In [9]:
others = df[df.HCPCS_Cd != 'A4253']
others.shape

(40208, 4)

In [10]:
dummy_df = pd.get_dummies(blood_glucose_test)

In [11]:
dummy_df.dtypes

Tot_Rfrg_Prvdrs           int64
Tot_Suplrs                int64
Avg_Suplr_Sbmtd_Chrg    float64
HCPCS_Cd_A4253            uint8
dtype: object

In [12]:
dummy_df.head()

,Tot_Rfrg_Prvdrs,Tot_Suplrs,Avg_Suplr_Sbmtd_Chrg,HCPCS_Cd_A4253
592,233762,43511,61.932717,1
1999,3093,1406,56.558110,1
2624,509,204,64.369821,1
3186,4387,2262,66.860627,1
3987,2258,924,49.084897,1


In [13]:
X = dummy_df.drop(columns='Avg_Suplr_Sbmtd_Chrg')
y = dummy_df['Avg_Suplr_Sbmtd_Chrg']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

In [15]:
X_train.shape, X_test.shape

((47, 3), (12, 3))

In [16]:
y_train.shape, y_test.shape

((47,), (12,))

In [17]:
X_train.head()

,Tot_Rfrg_Prvdrs,Tot_Suplrs,HCPCS_Cd_A4253
28314,8973,3846,1
31531,930,624,1
19393,5016,2355,1
8531,13999,7699,1
39428,470,251,1


In [18]:
X_test.dtypes

Tot_Rfrg_Prvdrs    int64
Tot_Suplrs         int64
HCPCS_Cd_A4253     uint8
dtype: object

In [19]:
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)